A extração dos itens foi "fatiada" em seções e categorias para facilitar e permitir que o processo seja realizado corretamente. Assim, são mais de um código, separados em categorias (como "Infantil" ou "Medicamentos", por exemplo), e as categorias são divididas em subdivisões ("Marcas", por exemplo), afim de dividir volumes grandes de produtos (já que grandes volumes podem demorar muito tempo para rodar, além de aumentar as chances de encontrar algum problema nas páginas e dificultar o encontro do mesmo).
Foi observado que os links mudam conforme as categorias, páginas carregadas e filtros aplicados.


Tem que extrair EAN (codigo do produto)

O código abaixo é capaz de extrair os nomes e preços dos produtos a partir do link, porém se limita aos produtos já carregados (15, no caso das páginas de categorias). Assim, é um passo inicial mas não é o que buscamos.

In [ ]:
#esse so carrega 750

import requests
from bs4 import BeautifulSoup
import time

# Adicionando um pequeno atraso entre as requisições
time.sleep(1)  # Atraso de 1 segundo

# URL base do site
base_url = 'https://www.drogariasminasmais.com.br/medicamentos'

# Número total de produtos e produtos por página
total_products = 800
products_per_page = 15

# Calculando o número total de páginas
total_pages = total_products // products_per_page + (1 if total_products % products_per_page != 0 else 0)

all_products = []

# Loop para percorrer todas as páginas
for page_num in range(1, total_pages + 1):
    # URL da página com a numeração
    url = f'{base_url}?page={page_num}'
    
    # Realizando a requisição para a página
    response = requests.get(url)
    
    if response.status_code != 200:
        print(f"Erro ao acessar a página {page_num}: {response.status_code}")
        continue
    
    # Usando BeautifulSoup para parsear o conteúdo
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Encontrando os elementos dos produtos na página
    product_name_elements = soup.find_all('span', class_='vtex-product-summary-2-x-productBrand')
    price_elements = soup.find_all('span', class_='vtex-product-price-1-x-sellingPriceValue')

    # Verificando se o número de produtos e preços coincide
    if len(product_name_elements) == len(price_elements):
        for name, price in zip(product_name_elements, price_elements):
            product_name = name.get_text(strip=True)
            price_value = price.get_text(strip=True)
            all_products.append({'name': product_name, 'price': price_value})
            print(f'Produto: {product_name} - Preço: {price_value}')
           
    else:
        print(f"Erro na página {page_num}: o número de produtos e preços não coincide.")

print(f"Total de produtos coletados: {len(all_products)}")


In [18]:
import requests
from bs4 import BeautifulSoup
import time

# Função para criar a URL com filtros e marca
def generate_url(category, brand, page_num):
    # Garantir que a marca esteja em minúsculas
    brand = brand.lower()  # Convertendo a marca para minúsculas
    
    # Monta a URL com base na categoria e marca
    url = f'https://www.drogariasminasmais.com.br/medicamentos/{category}/{brand}?initialMap=c&initialQuery=medicamentos&map=category-1,category-2,brand'
    
    # Adiciona a paginação
    return f'{url}&page={page_num}'

# Adicionando um pequeno atraso entre as requisições
time.sleep(1)  # Atraso de 1 segundo

# Categorias e Marcas
categories = [
    'mais-medicamentos', 'vitaminas', 'calmantes', 'analgesicos', 'gripe-e-resfriado', 
    'dor-e-febre', 'digestivo', 'colesterol-e-triglicerideos', 'anti-inflamatorios', 
    'alergias-e-infeccoes', 'relaxante-muscular', 'diabetes', 'hipertensao', 'fitoterapicos', 
    'anticoncepcionais'
]
brands = [
    'EMS', 'ACHE RX', 'EUROFARMA RX', 'EMS GENERICO', 'SIGMA PHARMA RX', 'APSEN', 
    'EUROFARMA GENERICO', 'LIBBS', 'BIOLAB RX', 'CIMED', 'MOMENTA', 'FARMOQUIMICA', 
    'SUPERA', 'MYRALIS', 'GERMED GENERICO', 'MEDLEY GENERICOS', 'ABBOTT RX', 'CRISTALIA/LATINOFARMA',
    'TORRENT', 'UNIAO QUIMICA GENOM', 'PRATI DONADUZZI GENERICO', 'PHARLAB', 'SANOFI RX', 
    'NOVARTIS RX', 'ACHE OTC', 'TEUTO GENERICO', 'MARJAN', 'ACNASE', 'BIOLAB GENERICO', 'GSK RX'
    # Adicione mais marcas conforme necessário
]


# Número total de produtos e produtos por página
total_products = 705  # Limite de 705 produtos
products_per_page = 15  # Número de produtos por página

# Calculando o número total de páginas
total_pages = total_products // products_per_page + (1 if total_products % products_per_page != 0 else 0)

all_products = []

# Loop para percorrer todas as categorias e marcas
for category in categories:
    for brand in brands:
        print(f"Buscando produtos para a categoria: {category} e marca: {brand}")
        
        # Loop para percorrer todas as páginas
        for page_num in range(1, total_pages + 1):
            # Gerar a URL para a página atual com as categorias, filtros e marca
            url = generate_url(category, brand, page_num)
            
            # Realizando a requisição para a página
            response = requests.get(url)
            
            if response.status_code != 200:
                print(f"Erro ao acessar a página {page_num}: {response.status_code}")
                continue
            
            # Usando BeautifulSoup para parsear o conteúdo
            soup = BeautifulSoup(response.content, 'html.parser')
            
            # Encontrando os elementos dos produtos na página
            product_name_elements = soup.find_all('span', class_='vtex-product-summary-2-x-productBrand')
            price_elements = soup.find_all('span', class_='vtex-product-price-1-x-sellingPriceValue')

            # Verificando se o número de produtos e preços coincide
            if len(product_name_elements) == len(price_elements):
                for name, price in zip(product_name_elements, price_elements):
                    product_name = name.get_text(strip=True)
                    price_value = price.get_text(strip=True)
                    all_products.append({'name': product_name, 'price': price_value})
                    print(f'Produto: {product_name} - Preço: {price_value}')
            else:
                print(f"Erro na página {page_num}: o número de produtos e preços não coincide.")
            
            # Limite de 705 produtos
            if len(all_products) >= total_products:
                break
        # Limite de 705 produtos
        if len(all_products) >= total_products:
            break

print(f"Total de produtos coletados: {len(all_products)}")


Buscando produtos para a categoria: mais-medicamentos e marca: EMS
Produto: Aglitil 30mg com 60 Comprimidos - Ems - Preço: R$109,78
Produto: Aglitil 15mg com 30 Comprimidos - Ems - Preço: R$64,55
Produto: Nesina Met 12,5 + 1000mg com 60 Comprimidos - Hypera - Preço: R$154,41
Produto: Nesina Pio 25mg + 30mg com 30 Comprimidos - Hypera - Preço: R$220,90
Produto: Nesina Pio 25mg + 15mg com 30 Comprimidos - Hypera - Preço: R$223,07
Produto: Nesina Met 12,5 + 850mg com 60 Comprimidos - Hypera - Preço: R$154,41
Produto: Nesina 12,5mg com 30 Comprimidos - Hypera - Preço: R$136,73
Produto: Maxsulid com 30 Comprimidos - Hypera - Preço: R$99,05
Produto: Predsim 20mg com 20 Comprimidos - Hypera - Preço: R$60,90
Produto: Predsim 40mg com 10 Comprimidos - Hypera - Preço: R$75,96
Produto: Gelol Gel Massageador - 60g - Preço: R$32,17
Produto: Tamarine Fibras Kids Sabor Morango - 240ml - Preço: R$71,13
Produto: Addera D3 5.000ui com 30 Comprimidos - Hypera - Preço: R$105,78
Produto: Bifilac - 30 Cápsu

aaaaaaaaaaaaaaaaaaa

In [ ]:
#mais medicamnentos

import requests
from bs4 import BeautifulSoup
import time

# Função para criar a URL com filtros e marca
def generate_url(filtro, brand, page_num):
    # Monta a URL com base em medicamentos, filtro (categoria) e marca
    url = f'https://www.drogariasminasmais.com.br/medicamentos/{filtro}/{brand}?initialMap=c&initialQuery=medicamentos&map=category-1,category-2,brand'
    # Adiciona a paginação
    return f'{url}&page={page_num}'

# Função para buscar os produtos de uma categoria e marca específica
def fetch_products_for_category_and_brand(filtro, brand):
    all_products = []  # Lista para armazenar todos os produtos encontrados
    page_num = 1  # Iniciar com a primeira página

    while True:
        # Gerar a URL para a página atual com o filtro e marca
        url = generate_url(filtro, brand, page_num)
        response = requests.get(url)
        
        if response.status_code != 200:
            print(f"Erro ao acessar a página {page_num}: {response.status_code}")
            break
        
        # Usando BeautifulSoup para parsear o conteúdo
        soup = BeautifulSoup(response.content, 'html.parser')

        # Encontrando os elementos dos produtos na página
        product_name_elements = soup.find_all('span', class_='vtex-product-summary-2-x-productBrand')
        price_elements = soup.find_all('span', class_='vtex-product-price-1-x-sellingPriceValue')

        # Se não houver produtos, paramos a execução
        if not product_name_elements:
            print(f'Não foram encontrados produtos na página {page_num}.')
            break
        
        # Exibindo produtos e preços
        for name, price in zip(product_name_elements, price_elements):
            product_name = name.get_text(strip=True)
            price_value = price.get_text(strip=True)
            all_products.append({'name': product_name, 'price': price_value})
            print(f'Produto: {product_name} - Preço: {price_value}')
        
        # Verificando se há mais páginas
        page_num += 1

    return all_products

# Função principal para buscar produtos por filtro e marca
def search_products(filtros, brands):
    for filtro in filtros:
        for brand in brands:
            print(f'\nBuscando produtos para o filtro: {filtro} e marca: {brand}')
            products = fetch_products_for_category_and_brand(filtro, brand)
            print(f'Total de produtos para {filtro} - {brand}: {len(products)}')

# Exemplos de filtros e marcas
filtros = [
    'mais-medicamentos', 'vitaminas', 'calmantes', 'analgesicos', 'gripe-e-resfriado', 
    'dor-e-febre', 'digestivo', 'colesterol-e-triglicerideos', 'anti-inflamatorios', 
    'alergias-e-infeccoes', 'relaxante-muscular', 'diabetes', 'hipertensao', 'fitoterapicos', 
    'anticoncepcionais'
]
brands = [
    'EMS', 'ACHE RX', 'EUROFARMA RX', 'EMS GENERICO', 'SIGMA PHARMA RX', 'APSEN', 
    'EUROFARMA GENERICO', 'LIBBS', 'BIOLAB RX', 'CIMED', 'MOMENTA', 'FARMOQUIMICA', 
    'SUPERA', 'MYRALIS', 'GERMED GENERICO', 'MEDLEY GENERICOS', 'ABBOTT RX', 'CRISTALIA/LATINOFARMA',
    'TORRENT', 'UNIAO QUIMICA GENOM', 'PRATI DONADUZZI GENERICO', 'PHARLAB', 'SANOFI RX', 
    'NOVARTIS RX', 'ACHE OTC', 'TEUTO GENERICO', 'MARJAN', 'ACNASE', 'BIOLAB GENERICO', 'GSK RX'
    # Adicione mais marcas conforme necessário
]

# Iniciar a busca
search_products(filtros, brands)


In [2]:
import requests
from bs4 import BeautifulSoup
import time

# Função para criar a URL com filtro (categoria) e marca
def generate_url(filtro, brand, page_num):
    # Monta a URL com base na categoria (filtro) e marca
    url = f'https://www.drogariasminasmais.com.br/medicamentos/{filtro}/{brand}?initialMap=c&initialQuery=medicamentos&map=category-1,category-2,brand'
    # Adiciona a paginação
    return f'{url}&page={page_num}'

# Função para buscar os produtos de uma categoria e marca específica
def fetch_products_for_category_and_brand(filtro, brand):
    all_products = []  # Lista para armazenar todos os produtos encontrados
    page_num = 1  # Iniciar com a primeira página

    while True:
        # Gerar a URL para a página atual com o filtro e marca
        url = generate_url(filtro, brand, page_num)
        response = requests.get(url)
        
        if response.status_code != 200:
            print(f"Erro ao acessar a página {page_num}: {response.status_code}")
            break
        
        # Usando BeautifulSoup para parsear o conteúdo
        soup = BeautifulSoup(response.content, 'html.parser')

        # Encontrando os elementos dos produtos na página
        product_name_elements = soup.find_all('span', class_='vtex-product-summary-2-x-productBrand')
        price_elements = soup.find_all('span', class_='vtex-product-price-1-x-sellingPriceValue')

        # Se não houver produtos, paramos a execução
        if not product_name_elements:
            print(f'Não foram encontrados produtos na página {page_num}.')
            break
        
        # Exibindo produtos e preços
        for name, price in zip(product_name_elements, price_elements):
            product_name = name.get_text(strip=True)
            price_value = price.get_text(strip=True)
            all_products.append({'name': product_name, 'price': price_value})
            print(f'Produto: {product_name} - Preço: {price_value}')
        
        # Verificando se há mais páginas
        page_num += 1

    return all_products

# Função principal para buscar produtos por categoria (filtro) e todas as marcas
def search_products_for_category(filtro, brands):
    print(f'\nBuscando produtos para a categoria: {filtro}')
    
    all_category_products = []  # Lista para armazenar todos os produtos de uma categoria
    
    # Loop para buscar todos os produtos de cada marca dentro da categoria
    for brand in brands:
        print(f'\nBuscando produtos para a marca: {brand} na categoria: {filtro}')
        products = fetch_products_for_category_and_brand(filtro, brand)
        all_category_products.extend(products)  # Adiciona os produtos dessa marca à lista geral
        print(f'Total de produtos encontrados para a categoria {filtro} - marca {brand}: {len(products)}')
    
    print(f'\nTotal de produtos encontrados para a categoria {filtro}: {len(all_category_products)}')

# Exemplos de filtros (categorias) e marcas
filtros = [
    'mais-medicamentos', 'vitaminas', 'calmantes', 'gripe-e-resfriado', 'analgésicos', 'similar', 'hipertensão', 'anti-inflamatórios', 
    'colesterol-e-triglicerídeos', 'anticoncepcionais', 'alergias e infecções', 'fitoterápicos', 'genéricos', 'dor-e-febre', 'digestivo', 
    'relaxante-muscular', 'diabetes', 'gastrite', 'emagrecedores', 'disfunção-erétil'
]

brands = [
    'ems', 'ache-rx', 'eurofarma-rx', 'ems-generico', 'sigma-pharma-rx', 'eurofarma-generico', 'cimed', 'biolab-rx', 'libbs', 'germed-generico',
    'apsen', 'medley-genericos', 'momenta', 'supera', 'farmoquimica', 'myralis', 'prati-donaduzzi-generico', 'pharlab', 'abbott-rx', 'torrent', 
    'uniao-quimica-genom', 'ache-otc', 'cristalia-latinofarma', 'teuto-generico', 'asepxia', 'acnase', 'sanofi-rx', 'neo-quimica-generico', 'biolab-generico', 
    'gsk-otc', 'novartis-rx', 'zanphy', 'marjan', 'neo-quimica-similar', 'procter-gambl', 'gsk-rx', 'nova-quimica-generico', 'merck-rx', 
    'airela', 'servier', 'allergan', 'uniao-quimica-generico', 'alcon', 'biolab-avert-otc', 'geolab-generico', 'zodiac-adium', 'sankyo-pharma', 
    'cellera', 'teuto-similar', 'divcom', 'bayer-rx', 'astrazeneca', 'legrand-generico', 'herbarium', 'biosintetica-generico', 'sigma-pharma-otc', 
    'megalabs', 'boehringer', 'bayer-otc', 'brace-pharma-rx', 'hertz-p-2', 'takeda', 'globo-generico', 'reckitt-benckiser-rb', 'cifarma', 'ranbaxy',
    'natulab', 'biosintetica', 'aspen', 'zambon', 'arovida', 'arese', 'viatris', 'sandoz-generico', 'pfizer', 'gross', 'ems-similar', 'lundbeck', 'jj-otc', 
    'cristalia', 'belfar', 'massime', 'hertz-p-1', 'geolab', 'galderma', 'cifarma-similar', 'chiesi-farmalab', 'janssen', 'feelclean', 'elseve', 'althaia', 
    'grunenthal', 'baldacci', 'abv', 'uniao-quimica', 'theraskin', 'besins', 'exeltis', 'torrent-generico', 'merck-generico', 'maxinutri', 'm-s-d', 
    'hebron-quesalon', 'gummy', 'delta', 'bausch-lomb', 'vitamedic-generico', 'nikkho-zydus', 'lilly', 'leopharma', 'pharmascience', 'medquimica-generico', 
    'diffucap-chemobras', 'usk-rx', 'multilab', 'moksha8', 'medquimica', 'm-s-d-organon', 'catarinense', 'boiron', 'nativita-generico', 'globo', 'astellas', 
    'unichems', 'hisamitsu', 'health-labs', 'gsk-stiefel', 'farmax', 'eurofarma-oaz', 'abbott-nutricional', 'weleda', 'vitafor', 'ucb-biopharma', 'sandoz-rx', 
    'bago', 'waldemiro', 'valeant', 'osorio-de-moraes', 'mundipharma', 'medley-rx', 'legrand-rx', 'arte-nativa', 'zydus-generico', 'wyeth-rx', 'vitamedic', 
    'united-rx', 'roche', 'natcofarma', 'caninha', 'blau', 'shire', 'prati-donaduzzi', 'glenmark', 'genomma', 'equaliv', 'diversos-similares', 'biolab-similar', 
    'bio-soft', 'almaroni', 'teva', 'philips-avente', 'organon', 'mylan', 'imec', 'herbamed', 'farmace-generico', 'farmabraz-beta-atalaia-ltda', 'eli-lilly', 
    'cifarma-generico', 'casa-granado', 'umuarama', 'tresemm', 'trb', 'semina', 'sanitall', 'pequenin', 'olvebra', 'novartis-otc', 'nova-quimica-similar', 
    'nestle-industrial', 'nativita-similar', 'minancora', 'millet-roux', 'melpoejo', 'melora', 'latinofarma', 'laboratorios-ferring', 'johnson-johnson-s-a', 
    'glenmark-generico', 'gigita', 'curatec', 'cheez-it', 'cazi', 'bunker', 'brasterapica', 'biomm', 'bianco', 'almeida-prado-ltda', 'abl-brasil'



    # Adicione mais marcas conforme necessário
]

# Executar a busca para uma categoria específica
category_to_search = 'mais-medicamentos'  # Escolha a categoria que deseja buscar
search_products_for_category(category_to_search, brands)



Buscando produtos para a categoria: mais-medicamentos

Buscando produtos para a marca: ems na categoria: mais-medicamentos
Produto: Aglitil 30mg com 60 Comprimidos - Ems - Preço: R$109,78
Produto: Aglitil 15mg com 30 Comprimidos - Ems - Preço: R$64,55
Produto: Nesina Met 12,5 + 1000mg com 60 Comprimidos - Hypera - Preço: R$154,41
Produto: Nesina Pio 25mg + 30mg com 30 Comprimidos - Hypera - Preço: R$220,90
Produto: Nesina Pio 25mg + 15mg com 30 Comprimidos - Hypera - Preço: R$223,07
Produto: Nesina Met 12,5 + 850mg com 60 Comprimidos - Hypera - Preço: R$154,41
Produto: Nesina 12,5mg com 30 Comprimidos - Hypera - Preço: R$136,73
Produto: Maxsulid com 30 Comprimidos - Hypera - Preço: R$99,05
Produto: Predsim 20mg com 20 Comprimidos - Hypera - Preço: R$60,90
Produto: Predsim 40mg com 10 Comprimidos - Hypera - Preço: R$75,96
Produto: Gelol Gel Massageador - 60g - Preço: R$32,17
Produto: Tamarine Fibras Kids Sabor Morango - 240ml - Preço: R$71,13
Produto: Addera D3 5.000ui com 30 Comprimid